In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


# PydanticOutputParser

1. Pydantic 모델 활용:
    - Python의 데이터 검증 및 설정 관리 위한 `Pydantic` 모델을 사용하여 출력 형식 정의
    - 가령 문자열, 정수, 리스트 등의 데이터 타입 및 유효성 검사 설정  
2. 출력 구조화:
    - 텍스트 응답을 JSON 형태와 같은 구조화된 데이터로 변환 
    - 변환된 데이터는 프로그래밍에 쉽게 활용 가능 
3. 에러 처리:
    - 출력이 정의된 포맷에 맞지 않을 경우 오류 반환 
    - 디버깅 하거나 응답 품질 개선에 활용

[Reference] https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.pydantic.PydanticOutputParser.html

In [11]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate 
from langchain.output_parsers import PydanticOutputParser
from langchain_teddynote.messages import stream_response
from itertools import chain

## 유효성 검증

In [5]:
# Pydantic 모델 정의
class WeatherResponse(BaseModel):
    location: str
    temperature: float
    condition: str

# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=WeatherResponse)

# 예제 출력
model_output = """{
    "location": "New York",
    "temperature": 25.4,
    "condition": "Sunny"
}"""

# 출력 파싱
parsed_output = parser.parse(model_output)
print(parsed_output)


location='New York' temperature=25.4 condition='Sunny'


In [7]:
# 예제 출력
model_output = """{
    "location": "New York",
    "temperature": "High",
    "condition": "Sunny"
}"""

# 출력 파싱
parsed_output = parser.parse(model_output)
print(parsed_output)

OutputParserException: Failed to parse WeatherResponse from completion {"location": "New York", "temperature": "High", "condition": "Sunny"}. Got: 1 validation error for WeatherResponse
temperature
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='High', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/float_parsing
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

## 포맷 정의

In [9]:
llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-4o-mini',
)

In [10]:
email_conversation = """From: 박민호 (minho.park@techsolarsystem.me)  
To: 정유진 (yujin.jung@greencityinnovations.me)  
Subject: 태양광 에너지 솔루션 협력 제안 및 미팅 일정  

안녕하세요, 정유진 팀장님,  

저는 테크솔라시스템의 박민호 부장입니다. 귀사가 최근 발표한 태양광 에너지 솔루션 "GreenSpark"에 대해 큰 관심을 가지고 있습니다. 테크솔라시스템은 신재생 에너지 설비 및 관련 기술 컨설팅에서 폭넓은 경험을 보유한 기업으로, 귀사의 혁신적인 솔루션과 협력하여 시너지를 창출할 수 있을 것이라 확신합니다.  

GreenSpark 솔루션에 대한 기술 자료를 요청드립니다. 특히 효율성 데이터, 설치 요구사항, 그리고 유지보수 프로세스에 대한 상세 정보를 보내주시면, 이를 바탕으로 저희가 제안할 협력 모델을 구체화할 수 있을 것입니다.  

더불어, 협력 가능성에 대해 논의하고자 다음 주 목요일(2월 8일) 오후 2시에 귀사 사무실에서의 미팅을 제안드립니다. 해당 일정이 가능하신지 확인 부탁드립니다.  

감사합니다.  

박민호  
부장  
테크솔라시스템 
"""

포맷을 정의하지 않는 경우...

In [12]:
prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

chain = prompt | llm 
answer = chain.stream({'email_conversation': email_conversation})
output = stream_response(answer, return_output=True)

중요 내용 요약:

- 발신자: 박민호 (테크솔라시스템 부장)
- 수신자: 정유진 (그린시티이노베이션 팀장)
- 주제: 태양광 에너지 솔루션 협력 제안 및 미팅 일정
- 관심 대상: 그린시티이노베이션의 태양광 에너지 솔루션 "GreenSpark"
- 요청 사항: GreenSpark 솔루션의 기술 자료 (효율성 데이터, 설치 요구사항, 유지보수 프로세스)
- 제안된 미팅 일정: 2월 8일 (목요일) 오후 2시, 귀사 사무실
- 확인 요청: 미팅 일정 가능 여부 확인

In [13]:
print(output)

중요 내용 요약:

- 발신자: 박민호 (테크솔라시스템 부장)
- 수신자: 정유진 (그린시티이노베이션 팀장)
- 주제: 태양광 에너지 솔루션 협력 제안 및 미팅 일정
- 관심 대상: 그린시티이노베이션의 태양광 에너지 솔루션 "GreenSpark"
- 요청 사항: GreenSpark 솔루션의 기술 자료 (효율성 데이터, 설치 요구사항, 유지보수 프로세스)
- 제안된 미팅 일정: 2월 8일 (목요일) 오후 2시, 귀사 사무실
- 확인 요청: 미팅 일정 가능 여부 확인


포맷 정의한 경우...

- description은 정확하게 명확하게 !!
- LLM이 작성된 description에서 필요 정보를 추출하여 처리

In [14]:
class EmailSummaryOutput(BaseModel):
    person: str = Field(description='메일 보낸 사람')
    email: str = Field(description='메일 보낸 사람의 이메일')
    subject: str = Field(description='메일 제목')
    summary: str = Field(description='메일 본문을 요약한 100자 이하의 텍스트')
    date: str = Field(description='메일 본문에 언급된 미팅 날짜와 시간')

In [17]:
# Output Parser 정의 
parser = PydanticOutputParser(pydantic_object=EmailSummaryOutput)

# instruction 확인 
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일 보낸 사람의 이메일", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 100자 이하의 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [18]:
# 프롬프트 정의 
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant.
Please answer the following questions in Korean. 

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

prompt = prompt.partial(
    format=parser.get_format_instructions()
)

In [20]:
# chain
chain = prompt | llm

# 답변 
response = chain.stream(
    {
        'email_conversation':email_conversation,
        'question':'이메일 내용에서 필요 내용을 정리해 주세요.',
    }
)

# 결과는 JSON 형태로 출력
output = stream_response(response, return_output=True)

```json
{
  "person": "박민호",
  "email": "minho.park@techsolarsystem.me",
  "subject": "태양광 에너지 솔루션 협력 제안 및 미팅 일정",
  "summary": "테크솔라시스템의 박민호가 정유진에게 GreenSpark 솔루션에 대한 기술 자료 요청 및 미팅 제안.",
  "date": "2024-02-08T14:00:00"
}
```

In [21]:
type(output)

str

In [22]:
# JSON 결과를 class 객체 형태로 추가 파싱 
output_structured = parser.parse(output)
print(output_structured)

person='박민호' email='minho.park@techsolarsystem.me' subject='태양광 에너지 솔루션 협력 제안 및 미팅 일정' summary='테크솔라시스템의 박민호가 정유진에게 GreenSpark 솔루션에 대한 기술 자료 요청 및 미팅 제안.' date='2024-02-08T14:00:00'


## 체인 구성 (with parser)

In [24]:
# chain
chain = prompt | llm | parser

# 답변 
response = chain.invoke(
    {
        'email_conversation':email_conversation,
        'question':'이메일 내용에서 필요 내용을 정리해 주세요.',
    }
)

# 결과 출력 
response

EmailSummaryOutput(person='박민호', email='minho.park@techsolarsystem.me', subject='태양광 에너지 솔루션 협력 제안 및 미팅 일정', summary='테크솔라시스템의 박민호가 정유진에게 GreenSpark 솔루션에 대한 기술 자료 요청 및 미팅 제안.', date='2024-02-08T14:00:00')

In [25]:
response.summary

'테크솔라시스템의 박민호가 정유진에게 GreenSpark 솔루션에 대한 기술 자료 요청 및 미팅 제안.'

## with_structured_output()

- 모델 정의 시 output parser를 함께 정의 
- 출력은 Pydantic 객체로 변환 가능 
- stream() 기능은 지원하지 않음

In [26]:
llm_with_structured = ChatOpenAI(
    temperature=0,
    model_name='gpt-4o-mini',
).with_structured_output(EmailSummaryOutput)

In [27]:
answer = llm_with_structured.invoke(email_conversation)
answer

EmailSummaryOutput(person='박민호', email='minho.park@techsolarsystem.me', subject='태양광 에너지 솔루션 협력 제안 및 미팅 일정', summary="태양광 에너지 솔루션 'GreenSpark'에 대한 협력 제안 및 미팅 일정 요청.", date='2024-02-08 14:00')

In [28]:
answer.summary

"태양광 에너지 솔루션 'GreenSpark'에 대한 협력 제안 및 미팅 일정 요청."

-----
** End of Documents **